In [1]:
import zarr
import obstore
import xarray as xr
import fsspec
from obstore.store import S3Store
from zarr.storage import ObjectStore

In [2]:
print(zarr.__version__)
print(obstore.__version__)

3.0.7.dev3+g9e8b50a
0.6.0


In [3]:
bucket = "nasa-veda-scratch"
path = "zarr-obstore-test/max/zarr-v3-obstore/test"
# Open store with obstore
s3store = S3Store(bucket, prefix=path)
store = ObjectStore(s3store)
# Write Zarr V3 data with obstore
z1 = zarr.create_array(
    store=store, shape=(10, 10), chunks=(5, 5), dtype="int32", overwrite=True
)
z1[:] = 42

In [4]:
bucket = "nasa-veda-scratch"
path = "zarr-obstore-test/max/zarr-v3-obstore/xarray-test"
# Open store with obstore
s3store = S3Store(bucket, prefix=path)
store = ObjectStore(s3store)

In [5]:
url = "https://nsf-ncar-era5.s3.amazonaws.com/e5.oper.an.pl/194106/e5.oper.an.pl.128_060_pv.ll025sc.1941060100_1941060123.nc"
encoding = {"PV": {"chunks": (1, 1, 721, 1440)}}

In [6]:
# Load data
ds = xr.open_dataset(fsspec.open(url).open(), engine="h5netcdf")
ds = ds.drop_encoding().isel(time=slice(0, 2))
dsl = ds.load()

In [7]:
store

ObjectStore(object_store://S3Store(bucket="nasa-veda-scratch", prefix="zarr-obstore-test/max/zarr-v3-obstore/xarray-test"))

In [8]:
dsl.to_zarr(store, consolidated=True, zarr_format=3, encoding=encoding, mode="w")

/opt/conda/lib/python3.11/site-packages/zarr/api/asynchronous.py:203: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(
